In [20]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import torch

from zunis.utils.config.loaders import get_default_integrator_config
from zunis.utils.config.loaders import create_integrator_args
from utils.integrands.camel import KnownSymmetricCamelIntegrand
from pprint import pprint

In [13]:
here = Path('.').resolve()
benchmarks = (here / '..').resolve()
print(benchmarks)
sys.path.append(str(benchmarks))

plt.rcParams.update({
    'font.size': 8,
    'text.usetex': True,
    'text.latex.preamble': r'\usepackage{amsfonts}',
    "figure.figsize": (6,4),
    'figure.dpi':150
})

sns.set_theme()
sns.set(font_scale=1.3)
sns.set_style("whitegrid")

/scratch/ndeutsch/zunis/zunis/experiments/benchmarks


In [14]:
from utils.data_storage.dataframe2sql import read_pkl_sql
from utils.config.loaders import get_sql_types
import sqlite3 as sql
from sqlalchemy import PickleType


In [15]:
con = sql.connect("benchmarks.db")
con.cursor().execute("SELECT name FROM sqlite_master where type = 'table'").fetchall()

[('camel_defaults',)]

In [18]:
dtypes = get_sql_types()
dtypes["value_history"] = PickleType

df = read_pkl_sql("benchmarks.db", "camel_defaults", dtypes=dtypes)
df.columns = df.columns.astype(str)
d_sigmas = pd.read_csv('../../exploration/gaussian_camel_integrands.csv')
d_sigma_camel = d_sigmas[['d','sigma_camel','sigma_1d','relative_std_camel']].rename(columns=
                                                                     {
                                                                         "sigma_camel":'s',
                                                                         'relative_std_camel':'relative_std'
                                                                     })

df = df.merge(d_sigma_camel,on=["d","s"],how="left")